In [3]:
#Importation des librairies
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

#Charger la base de données
DataComplete = pd.read_csv("C:/Users/sloua/Downloads/archive/base_finale.csv", sep=";")
data = np.array(DataComplete)

#Création de l'histogramme interactif
matrice4 = ([[ligne[1]] for ligne in data])
matrice2 = [[ligne[4], ligne[5], ligne[10], ligne[11], ligne[12], ligne[13], ligne[14], ligne[15]] for ligne in data]
matrice3 = np.array(matrice2)
matrice1 = np.array(matrice4)
matrice21 = np.array(matrice3)
app = dash.Dash(__name__)

#Choix des caractérstiques/variables retenues pour influencer la distribution des positionnements 
app.layout = html.Div([
    html.H1("Dashboard de Caractéristiques de Joueurs"),

    html.Label('Poids (en kg):'),
    dcc.Input(id='weight', type='number', value=70),
    
    html.Label('Taille (en cm):'),
    dcc.Input(id='height', type='number', value=180),

    html.Label('Vitesse (de 1 à 100):'),
    dcc.Input(id='pace', type='number', value=50),

    html.Label('Qualités de buteur (de 1 à 100):'),
    dcc.Input(id='shooting', type='number', value=50),

    html.Label('Qualité de passe (de 1 à 100):'),
    dcc.Input(id='passing', type='number', value=50),

    html.Label('Niveau en défense (de 1 à 100):'),
    dcc.Input(id='defending', type='number', value=50),

    html.Label('Force physique (de 1 à 100):'),
    dcc.Input(id='physic', type='number', value=50),

    html.Label('Qualité de dribble (de 1 à 100):'),
    dcc.Input(id='dribbling', type='number', value=50),

    html.Div(id='result-container'),
    dcc.Graph(id='poste-frequences'),
])

#Modifiaction des résultats (fréquence du positionnement) lorsqu'il y a une modification dans les caractéristiques
@app.callback(
    [Output('result-container', 'children'),
     Output('poste-frequences', 'figure')],
    [Input('weight', 'value'),
     Input('height', 'value'),
     Input('pace', 'value'),
     Input('shooting', 'value'),
     Input('passing', 'value'),
     Input('defending', 'value'),
     Input('physic', 'value'),
     Input('dribbling', 'value')]
)
#Fonction pour recommander un poste de football en fonction des caractéristiques physiques et des compétences d'un joueur
def update_result(weight, height, pace, shooting, passing, defending, physic, dribbling):
       
    liste2 = [weight, height, pace, shooting, passing, defending, physic, dribbling]
    
    distances = np.sqrt(np.sum(np.square(matrice21 - liste2), axis=1))
    indices_tries = np.argsort(distances)
    indices_proches = indices_tries[:30]
    valeurs1 = [matrice1[i][0] for i in indices_proches]
    
    liste = [val.split(", ") for val in valeurs1]
    freq = Counter([item for sublist in liste for item in sublist])
    noms = list(freq.keys())
    valeurs = list(freq.values())
    poste_le_plus_courant = max(liste, key=liste.count)

    if len(valeurs1) == len(poste_le_plus_courant):
        results = pd.DataFrame({
            "Joueur": valeurs1,
            "Poste": poste_le_plus_courant
        })
    else:
        results = pd.DataFrame()
    
    fig = {
        'data': [
            {'x': noms, 'y': valeurs, 'type': 'bar', 'name': 'Fréquence des postes'},
        ],
        'layout': {
            'title': 'Fréquence des postes',
            'xaxis': {'title': 'Poste'},
            'yaxis': {'title': 'Fréquence'},
        }
    }

    return results.to_string(), fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
    
    

